# Dji Processing

## Dependencies

Libraries needed for the program execution

In [1]:
import os
import rasterio
import glob
import numpy as np
import yaml

from rasterio.enums import Resampling
from dataclasses import asdict

from seadrone.data_structures import Profile, GeoreferencePartition, MergePartition, Partition, Resampling as ResamplingSeadrone
from seadrone.enums import SensorType, FlightMode
from seadrone.processing import FlightProcessor as processor, get_sensor

## Variables Set Up

**project_path**: It is used to define some useful subfolders of our flight and it's the folder path that contains all the flight data.

**file_path**: Is the full path of some flight capture that contains metadata like GPS, ...; It's used to generate automatically the sensor data needed for georeferencing, ...

**flight_folder**: Normally, this folder contains all the flight captures in .jpg format. The name can be whatever you want. For RGB should be RGB and for thermal data should be thermogram. But it's up to you.

**atygeo_folder**: This folder contains radiometric data processed by atygeo software. The name can be modified but normally it should be **atygeo**.

**georeference_out_folder**: This folder contains all the georefered captures. It's structure is "georeferences -> folder -> partition -> captures".

**merge_out_folder**: This folder contains all the merged captures. It's structure is "merges -> folder -> partition -> merge_method -> captures".

**resample_out_folder**: This folder contains all the resampled captures. It's structure is "resamples -> folder -> partition -> merge_method -> captures".

**metadata_out_name**: The full path where the metadata.csv file of each flight captures will be saved.

**fligth_lines_out_name**: The full path where the flight_lines.yaml file use for georeference and merge will be saved.

**sensor**: A representation of the sensor specifications of the flight like focal_length, bands_number, ...

**flight_profile**: The profile needed to save a jpg file using rasterio library.

**atygeo_profile**: The profile needed to save a radiometric file using rasterio library.

**flight_flip_axis** & **atygeo_flip_axis**: For georeference it's necessary to flip the data matrix [bands, height, width]. In DJI case, flipping the height is sufficient.

**steps_in_flight_line**: When processing a flight line, sometimes we want to skip some captures. Setting this variable to 1 means no skip, just process the next capture, 2 means skip 1 capture and so on.

**partitions**: Sometimes we don't want to include all flight lines in our mosaic. This dictionary defines some slices to process all, even and odd flight lines.

**partitions_for_georeference**: The flight lines to georeference.

**partitions_for_merge**: The flight lines to merge.

**partitions_for_resample**: The flight lines to resample.

**merge_methods**: The merge method to use. Available options are 'mean', 'first', 'min' and 'max'.

**resampling_methods**: Sometime we want to redude the spatial resolution of our mosaic. To achieve that we define a Resampling variable contained in the **data_structures.py** file that receives: The resampling method, the x scale, the y scale and a boolean indicating whether to downsample or upsample.

In [8]:
project_path = r'D:\DATOS1\Procesamientos\Texas\07_31_2022' # editable
file_path = r'd:\DATOS1\Procesamientos\Texas\07_31_2022\main\DJI_0001.JPG' # editable

flight_folder = os.path.join(project_path, 'main')
atygeo_folder = os.path.join(project_path, 'atygeo')

georeference_out_folder = os.path.join(project_path, 'georeferences')
merge_out_folder = os.path.join(project_path, 'merges')
resample_out_folder = os.path.join(project_path, 'resamples')

metadata_out_name = os.path.join(project_path, 'metadata', 'metadata.csv')
fligth_lines_out_name = os.path.join(project_path, 'flight_lines.yaml')

sensor = get_sensor(SensorType.DJI, file_path)

flight_profile = Profile(dtype = rasterio.uint8, count = sensor.bands_number, height = sensor.height, width = sensor.width, nodata = 0) # editable
atygeo_profile = Profile(dtype = rasterio.float32, count = 1, height = sensor.height, width = sensor.width, nodata = np.NAN) # editable

flight_flip_axis = [1] # editable
atygeo_flip_axis = [1] # editable

steps_in_flight_line = 2 # editable

partitions = {
    'even' : Partition('even', 0, None, 2),
    'odd' : Partition('odd', 1, None, 2),
    'all' : Partition('all', 0, None, 1),
}

partitions_for_georeference = ['even'] # editable
partitions_for_merge = ['even'] # editable
partitions_for_resample = ['even'] # editable
merge_methods = ['mean', 'first'] # editable
split_merge_bands = True

resampling_methods = [ # editable
    ResamplingSeadrone(Resampling.average, 5, 5, True),
    ResamplingSeadrone(Resampling.average, 10, 10, True),
    ResamplingSeadrone(Resampling.average, 15, 15, True),
]

## Processing

### Extract Metadata

In [ ]:
flight_metadata = processor.get_captures_metadata(flight_folder)
processor.save_metadata(flight_metadata, metadata_out_name)

### Load metadata

In [ ]:
flight_metadata = None

if os.path.exists(metadata_out_name):
    in_folder = os.path.dirname(metadata_out_name)
    in_name = os.path.basename(metadata_out_name)
    flight_metadata = processor.load_metadata(in_folder, in_name)

### Extract Flight Lines

**correction**: We normally have to reduce the flight angle by -90 o -270 because of the cameratransform library.

**altitude**: Average altitude of the drone during the flight. We usually set this to None. Meaning that we'll use the altitude of each capture for the georeferencing part.

**yaw**: Flight angle.

**pitch**: Average pitch of the drone during the flight. We usually set this to 0.

**roll**: Average roll of the drone during the flight. We usually set this to 0.

In [ ]:
correction = -90 # editable
altitude = None # editable
yaw = 90 # editable
pitch = 0 # editable
roll = 0 # editable

fligth_lines = processor.compute_flight_lines(flight_metadata.Yaw, altitude, yaw + correction, pitch, roll, FlightMode.OVERLAP_FIXED)
processor.save_flight_lines(fligth_lines, fligth_lines_out_name)

### Load Flight Lines

In [ ]:
fligth_lines = None

if os.path.exists(fligth_lines_out_name):
    in_folder = os.path.dirname(fligth_lines_out_name)
    in_name = os.path.basename(fligth_lines_out_name)
    fligth_lines = processor.load_flight_lines(in_folder, in_name)

### Georeference

#### Flight

In [ ]:
folders_to_georeference = [flight_folder]

for folder_to_georeference in folders_to_georeference:
    current_georeference_out_folder = os.path.join(georeference_out_folder, os.path.basename(folder_to_georeference))
    for partition_to_use in partitions_for_georeference:
        georefence_partition = GeoreferencePartition(**asdict(partitions[partition_to_use]), profile = flight_profile)
        processor.georefence_images(metadata = flight_metadata, partition = georefence_partition, flight_lines = fligth_lines,
                                    in_folder = folder_to_georeference, out_folder = current_georeference_out_folder,
                                    flip_axis = flight_flip_axis, use_metadata = True, overwrite = True)

#### Atygeo

In [ ]:
folders_to_georeference = [atygeo_folder]

for folder_to_georeference in folders_to_georeference:
    current_georeference_out_folder = os.path.join(georeference_out_folder, os.path.basename(folder_to_georeference))
    for partition_to_use in partitions_for_georeference:
        georefence_partition = GeoreferencePartition(**asdict(partitions[partition_to_use]), profile = atygeo_profile)
        processor.georefence_bands(metadata = flight_metadata, partition = georefence_partition, flight_lines = fligth_lines,
                                    in_folder = folder_to_georeference, out_folder = current_georeference_out_folder,
                                    flip_axis = atygeo_flip_axis, use_metadata = True, overwrite = True)

### Merge

**band_names**: If we set this variable using sensor.band_names or whatever we want, the mosaic will be splitted if the number of bands is the mosaic has the same length as band_names. In other case, the mosaic will not be splitted. The variable have the structure List[str]. Example: ['blue', 'red', ...]

In [ ]:
folders_to_merge = glob.glob(os.path.join(georeference_out_folder, '*'))
band_names = None

for folder_to_merge in folders_to_merge:
    current_merge_out_folder = os.path.join(merge_out_folder, os.path.basename(folder_to_merge))

    for partition_to_use in partitions_for_merge:
        for merge_method in merge_methods:
            merge_partition = MergePartition(**asdict(partitions[partition_to_use]), skip = steps_in_flight_line)
            processor.merge(metadata = flight_metadata, partition = merge_partition, flight_lines = fligth_lines,
                            in_folder = folder_to_merge, out_folder = current_merge_out_folder, method = merge_method,
                            band_names = band_names)

### Resample

In [9]:
folders_to_resample = glob.glob(os.path.join(merge_out_folder, '*'))

for folder_to_resample in folders_to_resample:
    processor.resample(folder_to_resample, resample_out_folder, resampling_methods)